In [1]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical
import numpy as np
import os

print("ok")

Using TensorFlow backend.


ok


In [2]:
MAX_SEQUENCE_LENGTH = 80
MAX_NB_WORDS = 10000
EMBEDDING_DIM = 100
VALIDATION_SPLIT = 0.2
TEST_SPLIT=0.2 # my set

texts = []
labels = []
texts_raw=[]
f1=open("D:\\data\\wangli\\pos_fenci.txt",errors="ignore")
lines1=f1.readlines();
for line in lines1:
    _item=line.strip();
    texts.append(_item);
    texts_raw.append(_item)
    labels.append(1);
    
f2=open("d:\\data\\wangli\\neg_fenci.txt",errors="ignore")
lines2=f2.readlines();
for line in lines2:
    _item=line.strip();
    texts.append(_item);
    texts_raw.append(_item)
    labels.append(0);
    
f3=open("d:\\data\\wangli\\dm_fenci.txt",errors='ignore')
lines3=f3.readlines()
for line in lines3:
    _item=line.strip();
    texts_raw.append(_item)
    
f4=open("d:\\data\\wangli\\rw_fenci.txt",errors='ignore')
lines4=f4.readlines()
for line in lines4:
    _item=line.strip()
    texts_raw.append(_item)
    
print("ok")

ok


In [3]:
tokenizer = Tokenizer(nb_words=MAX_NB_WORDS)
tokenizer.fit_on_texts(texts_raw)
sequences = tokenizer.texts_to_sequences(texts)


word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)

labels = to_categorical(np.asarray(labels))
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', labels.shape)

indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]
nb_validation_samples = int(0.6 * data.shape[0])


x_train = data[:nb_validation_samples]
y_train = labels[:nb_validation_samples]

x_val = data[nb_validation_samples:nb_validation_samples+int(0.2*data.shape[0])]
y_val = labels[nb_validation_samples:nb_validation_samples+int(0.2*data.shape[0])]

x_test=data[nb_validation_samples+int(0.2*data.shape[0]):]
y_test=labels[nb_validation_samples+int(0.2*data.shape[0]):]



print("ok")

C:\Users\qct20\Anaconda2\envs\gpu_env\lib\site-packages\keras\preprocessing\text.py:145: UserWarning: The `nb_words` argument in `Tokenizer` has been renamed `num_words`.
  warnings.warn('The `nb_words` argument in `Tokenizer` '


Found 79598 unique tokens.
Shape of data tensor: (21124, 80)
Shape of label tensor: (21124, 2)
ok


In [4]:
GLOVE_DIR = "D:\\data"
embeddings_index = {}
f = open(os.path.join(GLOVE_DIR, 'vector.txt'),encoding='utf-8',errors="ignore")
for line in f:
    values = line.split(' ')
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Total %s word vectors in Glove 6B 100d.' % len(embeddings_index))

embedding_matrix = np.random.random((len(word_index) + 1, EMBEDDING_DIM))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

print("ok")

Total 98625 word vectors in Glove 6B 100d.
ok


In [5]:
from keras.layers import Embedding
embedding_layer = Embedding(len(word_index) + 1,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=True)

In [6]:
from keras.models import Sequential
from keras.layers import Embedding, Flatten, Dense
from keras import layers

model = Sequential()
model.add(embedding_layer)
#model.add(Flatten())
model.add(layers.LSTM(128))#,return_sequences=True)
model.add(layers.Dropout(0.5));
#model.add(layers.LSTM(128))
#model.add(layers.Dropout(0.5));
model.add(Dense(2, activation='sigmoid'))





Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [7]:
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['acc'])
history = model.fit(x_train, y_train,
                    epochs=20,
                    batch_size=128,
                    validation_data=(x_val, y_val))


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Train on 12674 samples, validate on 4224 samples
Epoch 1/20
12674/12674 [==============================] - 8s - loss: 0.4735 - acc: 0.7832 - val_loss: 0.3434 - val_acc: 0.8653
Epoch 2/20
12674/12674 [==============================] - 7s - loss: 0.3390 - acc: 0.8663 - val_loss: 0.4144 - val_acc: 0.8163
Epoch 3/20
12674/12674 [==============================] - 7s - loss: 0.2856 - acc: 0.8874 - val_loss: 0.2756 - val_acc: 0.8951
Epoch 4/20
12674/12674 [==============================] - 7s - loss: 0.2481 - acc: 0.9020 - val_loss: 0.3686 - val_acc: 0.8504
Epoch 5/20
12674/12674 [==============================] - 7s - loss: 0.2132 - acc: 0.9188 - val_loss: 0.2508 - val_acc: 0.9060
Epoch 6/20
12674/12674 [==============================] - 7s - loss: 0.1834 - acc: 0.9329 - val_loss: 0

# 以下是分析和预测

In [8]:
import matplotlib.pyplot as plt

acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1, len(acc) + 1)

plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

<Figure size 640x480 with 1 Axes>

<Figure size 640x480 with 1 Axes>

In [9]:
results=model.predict(x_test)
from numpy import argmax
y_pre = argmax(results,axis=1) 
print("ok")

ok


In [10]:
print(y_pre[:10])

[1 1 0 1 1 0 0 1 1 0]


In [11]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score;
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score;
lst1=[]#true,in fact
for i in range(0,len(y_test)):
   
    if y_test[i][0]==0 and y_test[i][1]==1:
        lst1.append(1);
    if y_test[i][0]==1 and y_test[i][1]==0:
        lst1.append(0);
   
yy_test=np.array(lst1)
p=precision_score(y_pre,yy_test,average='macro')
r=recall_score(y_pre,yy_test,average='macro')
f1=f1_score(y_pre,yy_test,average='macro')
print(p)
print(r)
print(f1)

print("以下是IMRD部分")
dic=dict()
for i in range(0,len(yy_test)):
    if yy_test[i] not in dic:
        lst_small=list();
        lst_small.append(y_pre[i])
        dic[yy_test[i]]=lst_small;
    else:
        dic[yy_test[i]].append(y_pre[i])
dic=dict(sorted(dic.items(),key=lambda x:x[0]))

for ee in dic:
    lst_sub=list();
    for j in range(0,len(dic[ee])):
        lst_sub.append(ee)
    print(ee)
    
    print(accuracy_score(np.array(lst_sub),np.array(dic[ee])))
    p=precision_score(np.array(lst_sub),np.array(dic[ee]),average='macro');
    r=recall_score(np.array(lst_sub),np.array(dic[ee]),average='macro');
    f1=f1_score(np.array(lst_sub),np.array(dic[ee]),average='macro');
   
    print(p)
    print(r)
    print (f1)
        

0.9135638741702199
0.9137066791193937
0.9136081949137216
以下是IMRD部分
0
0.907221425155428
0.5
0.453610712577714
0.47567703109327986
1
0.9199063231850118
0.5
0.4599531615925059
0.47914125396438156


C:\Users\qct20\Anaconda2\envs\gpu_env\lib\site-packages\sklearn\metrics\classification.py:1145: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
C:\Users\qct20\Anaconda2\envs\gpu_env\lib\site-packages\sklearn\metrics\classification.py:1145: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
